In [ ]:
!pip install -q google-generativeai gtts Speechrecognition pydub python-docx pdfkit
!sudo apt-get install -y wkhtmltopdf


In [ ]:
import google.generativeai as genai
import ipywidgets as widgets
from IPython.display import display, HTML, Markdown
import json
import speech_recognition as sr
from gtts import gTTS
import pdfkit
from docx import Document
import os


In [ ]:

GEMINI_API_KEY = "Enter your Gemini API key here"
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel("gemini-1.5-flash")

In [ ]:
def record_voice():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        audio = r.listen(source)
    try:
      return r.recognize_google(audio)
    except sr.UnknownValueError:
        return "Could not understand audio."

In [ ]:
def process_sentence(sentence, tone="Neutral", word_count=None):
    prompt = f"""Improve this text with:
1. Grammar/punctuation corrections
2. Rephrase in a {tone} tone
3. Suggest synonyms for weak words
{f"4. Adjust to {word_count} words" if word_count else ""}
Text: \"{sentence}\""""
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
def process_text(text):
    prompt = f"""Improve this text with:
1. Grammar and punctuation corrections
2. Rephrase for clarity and fluency
3. Suggest vocabulary improvements
Text: "{text}"
"""
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"Error: {str(e)}"


In [ ]:
input_mode = widgets.ToggleButtons(
    options=['Text Input', 'File Upload', 'Voice Input'],
    description='Input Mode:',
    button_style='info'  # 'success', 'info', 'warning', 'danger' or ''
)

In [ ]:
text_input = widgets.Textarea(
    placeholder='Enter text or click 🎤 to speak...',
    layout=widgets.Layout(width='100%', height='100px')
)

In [ ]:
voice_button = widgets.Button(
    description="Record Voice",
    button_style="warning",
    tooltip="Click to record your voice"
)

In [ ]:
file_upload = widgets.FileUpload(
    accept='.txt',
    multiple=False,
    description='📁 Upload .txt File'
)

In [ ]:
process_button = widgets.Button(
    description="Enhance Text",
    button_style="success"
)
export_txt = widgets.Button(description="Export TXT")
export_pdf = widgets.Button(description="Export PDF")
export_docx = widgets.Button(description="Export DOCX")

In [ ]:
input_container = widgets.VBox()
def update_input_mode(change):
    if change['new'] == 'Text Input':
        input_container.children = [text_input]
    elif change['new'] == 'Voice Input':
        input_container.children = [voice_button]
    else:
        input_container.children = [file_upload]
input_mode.observe(update_input_mode, names='value')
update_input_mode({'new': input_mode.value})
export_panel = widgets.HBox([
    export_txt,
    export_pdf,
    export_docx
])
output_display = widgets.Output(
    layout=widgets.Layout(
        border='2px solid #2ecc71',
        padding='10px'
    )
)

In [ ]:
def on_process_click(b):
    output_display.clear_output()

    with output_display:
        text = ""

        if input_mode.value == 'Text Input':
            text = text_input.value

        elif input_mode.value == 'Voice Input':
            text = record_voice()

        else:  # File Upload
            if file_upload.value:
                uploaded = list(file_upload.value.values())[0]
                text = uploaded['content'].decode('utf-8')

        if not text.strip():
            display(HTML("<b style='color:red'>▲ Please provide input</b>"))
            return

        enhanced = process_text(text)
        display(Markdown(f"*Enhanced Text:*\n\n{enhanced}"))
process_button.on_click(on_process_click)
display(HTML("<h1 style='color: #2c3e50'>AI Writing Assistant</h1>"))

display(widgets.VBox([
    input_mode,
    input_container,
    process_button,
    output_display
]))